In [74]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

from medicine import Medicine

def convertToDF(dataList):
    return pd.DataFrame(
        data = {
            'updateAt': list(map(lambda x: x.updateAt, dataList)),
            'type': list(map(lambda x: x.type, dataList)),
            'date': list(map(lambda x: x.date, dataList)),
            'name': list(map(lambda x: x.name, dataList)),
            'sheet': list(map(lambda x: x.sheet, dataList)),
            'unit': list(map(lambda x: x.unit, dataList)),
            'housou': list(map(lambda x: x.housou, dataList))
    }
)

In [75]:
def getMedicine(td):
    length =  len(re.split('\d+', td[4].text.splitlines()[1])[0])
    med = Medicine(
        updateAt=td[0].text,
        type = td[2].text.splitlines()[0],
        date = td[1].text.splitlines()[0],
        name = td[4].text.splitlines()[0],
        sheet = re.split('\d+', td[4].text.splitlines()[1])[0],
        unit = td[4].text.splitlines()[1][length:],
        housou = '',
        )
    return med

In [76]:
def getDfFromUrl(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    tr = soup.find_all('tr', class_='result_table')

    dataList = []

    for tr in tr:
        td = tr.find_all('td')
        med = getMedicine(td)
        dataList.append(med)
        
    return convertToDF(dataList)

In [126]:
def getEndpoints():
    res = requests.get('https://drugshortage.jp/index.php')
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.findAll('table')
    col = table[2]

    endpoint = list(map(lambda x: x['href'], col.findAll('a') )) 
    endpoint = list(filter(lambda x: 'yago' in x , endpoint))
    endpoint = list(map(lambda x: 'https://drugshortage.jp/index.php' + x, endpoint))

    return endpoint

In [72]:
# url を指定するとDFを返すメソッド
# df = getDfFromUrl('https://drugshortage.jp/index.php?yago=%E5%B1%8B%E5%8F%B7%E7%84%A1%E3%81%97')
# df.to_csv('all1.csv', index=False)

In [127]:
endpoints = getEndpoints()

In [146]:
# 3秒おきにデータを取得してdfに追加していく
count = 0
df = []

for url in endpoints:
    count+=1
    rate = (count / len(endpoints)) * 100
    print(rate)
    time.sleep(3)

    try:
        getDf = getDfFromUrl(url)
        df.append(getDf)
        # pd.concat([df, getDf], axis=0)
    except:
        print('error')
        pass

0.7246376811594203
1.4492753623188406
2.1739130434782608
2.898550724637681
3.6231884057971016
4.3478260869565215
5.072463768115942
5.797101449275362
6.521739130434782
7.246376811594203
7.971014492753622
8.695652173913043
9.420289855072465
10.144927536231885
10.869565217391305
11.594202898550725
12.318840579710146
13.043478260869565
13.768115942028986
14.492753623188406
15.217391304347828
15.942028985507244
16.666666666666664
17.391304347826086
18.115942028985508
18.84057971014493
19.565217391304348
20.28985507246377
21.014492753623188
21.73913043478261
22.463768115942027
23.18840579710145
23.91304347826087
24.637681159420293
25.36231884057971
26.08695652173913
26.811594202898554
27.536231884057973
28.26086956521739
28.985507246376812
29.71014492753623
30.434782608695656
31.15942028985507
31.88405797101449
32.608695652173914
33.33333333333333
34.05797101449276
34.78260869565217
35.507246376811594
36.231884057971016
36.95652173913043
37.68115942028986
38.405797101449274
39.13043478260869

In [147]:
df

[         updateAt  type         date              name sheet     unit housou
 0     2021年11月24日  販売中止  2022年05月01日      ソランタール錠１００ｍｇ   ＰＴＰ    1000錠       
 1     2021年11月24日  販売中止  2022年03月11日         ノイロビタン配合錠   ＰＴＰ    1000錠       
 2     2021年11月24日  出荷調整  2021年11月21日       ロペミンカプセル１ｍｇ   ＰＴＰ  100カプセル       
 3     2021年11月24日  出荷調整  2021年11月21日       ロペミンカプセル１ｍｇ   ＰＴＰ  500カプセル       
 4     2021年11月19日  販売中止  2022年04月01日         フェアストン錠４０   ＰＴＰ     100錠       
 ...           ...   ...          ...               ...   ...      ...    ...
 1746  2020年01月06日  販売中止  2020年06月01日        ヨウリダモール錠２５   ＰＴＰ     100錠       
 1747  2019年12月16日  販売中止  2019年12月16日  アルシオドールカプセル０．５μｇ   ＰＴＰ  100カプセル       
 1748  2019年12月16日  販売中止  2019年12月16日    アルシオドールカプセル１μｇ   ＰＴＰ  100カプセル       
 1749  2019年05月08日  販売中止  2019年05月08日   エスエーワン配合カプセルＴ２０   ＰＴＰ  140カプセル       
 1750  2019年05月08日  販売中止  2019年05月08日   エスエーワン配合カプセルＴ２５   ＰＴＰ  140カプセル       
 
 [1751 rows x 7 columns],
       updateAt  type         date  

In [148]:
data = pd.concat(df, axis=0)

In [149]:
data

,updateAt,type,date,name,sheet,unit,housou
0,2021年11月24日,販売中止,2022年05月01日,ソランタール錠１００ｍｇ,ＰＴＰ,1000錠,
1,2021年11月24日,販売中止,2022年03月11日,ノイロビタン配合錠,ＰＴＰ,1000錠,
2,2021年11月24日,出荷調整,2021年11月21日,ロペミンカプセル１ｍｇ,ＰＴＰ,100カプセル,
3,2021年11月24日,出荷調整,2021年11月21日,ロペミンカプセル１ｍｇ,ＰＴＰ,500カプセル,
4,2021年11月19日,販売中止,2022年04月01日,フェアストン錠４０,ＰＴＰ,100錠,
...,...,...,...,...,...,...,...
51,2020年04月13日,販売中止,2020年06月01日,クロピドグレル錠７５ｍｇ「ＺＥ」,ＰＴＰ,500錠,
52,2020年04月07日,販売中止,2021年07月01日,ゾルピデム酒石酸塩錠５ｍｇ「ＺＥ」,ＰＴＰ,140錠,
53,2020年04月06日,販売中止,2020年04月06日,アンブロキソール塩酸塩徐放カプセル４５ｍｇ「ＺＥ」,ＰＴＰ,100カプセル,
54,2020年04月06日,販売中止,2020年04月06日,アンブロキソール塩酸塩徐放カプセル４５ｍｇ「ＺＥ」,ＰＴＰ,500カプセル,


In [150]:
data.to_csv('all.csv', index=False)